In [ ]:
!pip install MediaPipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pr

In [ ]:
# Nama File: preprocess_landmarks.py

import os
import glob
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import google.colab.drive as drive
drive.mount('/content/drive')

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


Mounted at /content/drive


In [ ]:
import time
import pickle
import shutil
from pathlib import Path

# --- KONFIGURASI ---
METODE = 'BISINDO' # Pilih 'SIBI' atau 'BISINDO'

if METODE == 'SIBI':
    INPUT_DIR = '/content/drive/MyDrive/Skripsi/dataset/SIBI_augmentend'
    DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/Skripsi/dataset/SIBI_landmarks'
    MAX_HANDS = 1
else:
    INPUT_DIR = '/content/drive/MyDrive/Skripsi/dataset/BISINDO'
    DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/Skripsi/dataset/BISINDO_landmarks'
    MAX_HANDS = 2

# Simpan ke local dulu untuk menghindari masalah koneksi Drive
LOCAL_OUTPUT_DIR = '/content/temp_landmarks'
CHECKPOINT_FILE = '/content/checkpoint_landmarks.pkl'

NUM_LANDMARKS = 42 * 3  # 21 landmarks * 3 (x,y,z) * 2 tangan
CHECKPOINT_INTERVAL = 50  # Simpan checkpoint setiap N file

# --- Inisialisasi MediaPipe ---
mp_hands = mp.solutions.hands
hands_model = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=MAX_HANDS,
    min_detection_confidence=0.5
)

def extract_landmarks(image_path):
    """Membaca gambar, mengekstrak, dan mengembalikan landmark."""
    try:
        image = cv2.imread(image_path)
        if image is None:
            return None

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands_model.process(image_rgb)

        landmarks_vector = np.zeros(NUM_LANDMARKS, dtype=np.float32)
        if results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[i].classification[0].label
                coords = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                relative_coords = (coords - coords[0]).flatten()

                if handedness == 'Right':
                    landmarks_vector[0:63] = relative_coords
                elif handedness == 'Left':
                    landmarks_vector[63:126] = relative_coords

        return landmarks_vector
    except Exception as e:
        print(f"\n⚠️ Error saat ekstrak {image_path}: {e}")
        return None

def save_with_retry(output_path, data, max_retries=3):
    """Simpan file dengan mekanisme retry jika gagal."""
    for attempt in range(max_retries):
        try:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            np.save(output_path, data)
            return True
        except (ConnectionAbortedError, OSError, IOError) as e:
            if attempt < max_retries - 1:
                print(f"\n⚠️ Gagal menyimpan {os.path.basename(output_path)}, retry {attempt+1}/{max_retries}...")
                time.sleep(2)  # Tunggu 2 detik sebelum retry
            else:
                print(f"\n❌ Gagal menyimpan {output_path} setelah {max_retries} percobaan: {e}")
                return False
    return False

def save_checkpoint(processed_files):
    """Simpan checkpoint ke file."""
    try:
        with open(CHECKPOINT_FILE, 'wb') as f:
            pickle.dump(processed_files, f)
    except Exception as e:
        print(f"\n⚠️ Gagal menyimpan checkpoint: {e}")

def load_checkpoint():
    """Load checkpoint jika ada."""
    if os.path.exists(CHECKPOINT_FILE):
        try:
            with open(CHECKPOINT_FILE, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"\n⚠️ Gagal load checkpoint: {e}")
            return set()
    return set()

# --- PROSES UTAMA ---
print(f"🚀 Memulai pra-pemrosesan untuk dataset: {METODE}")
print(f"📂 Membaca gambar dari: {INPUT_DIR}")
print(f"💾 Menyimpan landmarks ke local: {LOCAL_OUTPUT_DIR}")
print(f"☁️  Akan dipindahkan ke Drive: {DRIVE_OUTPUT_DIR}\n")

# Buat direktori output lokal
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

# Ambil semua path gambar
image_paths = sorted(glob.glob(os.path.join(INPUT_DIR, '*/*.jpg')))
if not image_paths:
    raise ValueError(f"❌ Tidak ada gambar ditemukan di {INPUT_DIR}")

print(f"📊 Total gambar ditemukan: {len(image_paths)}")

# Load checkpoint jika ada
processed_files = load_checkpoint()
if processed_files:
    print(f"♻️  Melanjutkan dari checkpoint: {len(processed_files)} file sudah diproses\n")
    # Filter gambar yang belum diproses
    image_paths = [p for p in image_paths if p not in processed_files]
    print(f"📋 Sisa gambar yang perlu diproses: {len(image_paths)}\n")

failed_files = []
processed_count = len(processed_files)

for idx, image_path in enumerate(tqdm(image_paths, desc="Mengekstrak Landmarks")):
    # Ekstrak landmarks
    landmarks = extract_landmarks(image_path)

    if landmarks is not None:
        # Buat path output lokal
        relative_path = os.path.relpath(image_path, INPUT_DIR)
        output_path = os.path.join(LOCAL_OUTPUT_DIR, relative_path)
        output_path = os.path.splitext(output_path)[0] + '.npy'

        # Simpan landmarks dengan retry
        if save_with_retry(output_path, landmarks):
            processed_files.add(image_path)
            processed_count += 1
        else:
            failed_files.append(image_path)
    else:
        print(f"\n⚠️ Gagal ekstrak landmarks dari: {image_path}")
        failed_files.append(image_path)

    # Simpan checkpoint secara berkala
    if (idx + 1) % CHECKPOINT_INTERVAL == 0:
        save_checkpoint(processed_files)

# Simpan checkpoint terakhir
save_checkpoint(processed_files)

hands_model.close()

print("\n" + "="*60)
print("✅ Ekstraksi landmarks selesai!")
print(f"📊 Total berhasil diproses: {processed_count}/{len(image_paths) + len(processed_files)}")

if failed_files:
    print(f"\n⚠️  {len(failed_files)} file gagal diproses:")
    for f in failed_files[:10]:  # Tampilkan 10 pertama
        print(f"   - {os.path.basename(f)}")
    if len(failed_files) > 10:
        print(f"   ... dan {len(failed_files) - 10} file lainnya")

# --- PINDAHKAN KE GOOGLE DRIVE ---
print("\n" + "="*60)
print("📦 Memindahkan file ke Google Drive...")
print(f"Dari: {LOCAL_OUTPUT_DIR}")
print(f"Ke  : {DRIVE_OUTPUT_DIR}")

try:
    # Buat direktori Drive jika belum ada
    os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

    # Copy dengan progress
    total_files = sum([len(files) for _, _, files in os.walk(LOCAL_OUTPUT_DIR)])
    print(f"Total file untuk dipindahkan: {total_files}")

    # Gunakan shutil.copytree dengan dirs_exist_ok=True
    shutil.copytree(LOCAL_OUTPUT_DIR, DRIVE_OUTPUT_DIR, dirs_exist_ok=True)

    print("✅ File berhasil dipindahkan ke Google Drive!")

    # Hapus file lokal dan checkpoint
    print("\n🧹 Membersihkan file temporary...")
    shutil.rmtree(LOCAL_OUTPUT_DIR)
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)
    print("✅ Cleanup selesai!")

except Exception as e:
    print(f"\n❌ Error saat memindahkan ke Drive: {e}")
    print(f"⚠️  File tetap tersimpan di: {LOCAL_OUTPUT_DIR}")
    print("💡 Anda bisa mencoba copy manual atau jalankan ulang proses ini")

print("\n" + "="*60)
print("🎉 Pra-pemrosesan selesai!")
print("="*60)

🚀 Memulai pra-pemrosesan untuk dataset: BISINDO
📂 Membaca gambar dari: /content/drive/MyDrive/Skripsi/dataset/BISINDO
💾 Menyimpan landmarks ke local: /content/temp_landmarks
☁️  Akan dipindahkan ke Drive: /content/drive/MyDrive/Skripsi/dataset/BISINDO_landmarks

📊 Total gambar ditemukan: 10400


Mengekstrak Landmarks: 100%|██████████| 10400/10400 [57:42<00:00,  3.00it/s]



✅ Ekstraksi landmarks selesai!
📊 Total berhasil diproses: 10400/20800

📦 Memindahkan file ke Google Drive...
Dari: /content/temp_landmarks
Ke  : /content/drive/MyDrive/Skripsi/dataset/BISINDO_landmarks
Total file untuk dipindahkan: 10400

❌ Error saat memindahkan ke Drive: [('/content/temp_landmarks/V/flipped_197.npy', '/content/drive/MyDrive/Skripsi/dataset/BISINDO_landmarks/V/flipped_197.npy', "[Errno 5] Input/output error: '/content/drive/MyDrive/Skripsi/dataset/BISINDO_landmarks/V/flipped_197.npy'")]
⚠️  File tetap tersimpan di: /content/temp_landmarks
💡 Anda bisa mencoba copy manual atau jalankan ulang proses ini

🎉 Pra-pemrosesan selesai!


In [ ]:
SAMPLE_LIMIT = 15
VISUALIZE_SAMPLE = True
SAMPLE_LIMIT = None

if VISUALIZE_SAMPLE:
  if SAMPLE_LIMIT is None:
    should_visualize = True
  else:
    if class_name not in class_sample_count:
      class_sample_count[class_name] = 0
    if class_sample_count[class_name] < SAMPLE_LIMIT:
      should_visualize = True
      class_sample_count[class_name] += 1
    else:
      should_visualize = False